# 現代制御
## Topics
* システムの状態方程式表現
* 状態遷移行列
* 安定性
* 可制御性
* 可観測性
* フィードバック
* 最適レギュレータ
* サーボ系
* オブザーバ

## システムの状態方程式表現
状態，入力，出力ベクトルはそれぞれ, $x(t) \in R^n, u(t) \in R^m, y(t) \in R^r$ とし，
定数行列$A \in R^{n\times n}, B \in R^{n\times m}, C \in R^{r\times n}$によって，次のような線形システムが表現される．

$$
\begin{array}{}
\dot{x} &= Ax(t) + Bu(t) \\
y &= Cx(t)
\end{array}
$$


In [18]:
import numpy as np
class LinearSystem:
    def __init__(self, A, B, C):
        self.A = A
        self.B = B
        self.C = C
        
    def dot(self, x, u):
        return self.A*x + self.B*u
    
    def output(self, x):
        return self.C*x
    

        

## 状態遷移行列
行列指数関数$e^{At}$を状態遷移行列と呼ぶ．その定義は，
$$
e^{At} = I + At + \frac{1}{2!}A^2t^2 + \cdots + \frac{1}{k!}A^kt^k + \cdots
$$
となる．状態遷移行列を得る主な方法はラプラス逆変換によるものである．

In [19]:
def StateTransitionMat(A, t, k=20):
    # Approximation method
    E = np.identity(A.shape[0]) # identity term
    a = np.identity(A.shape[0])
    for i in range(1,k+1):
        a = a*A*t/float(i)
        E += a
        
    return E

A = np.array([[1,0],[0,1]])
print(StateTransitionMat(A, 0.01, 1))
print(StateTransitionMat(A, 0.01, 10))
print(StateTransitionMat(A, 0.01, 100))
print(StateTransitionMat(A, 0.01, 1000))
       
print(StateTransitionMat(A, 0.1, 1000))
print(StateTransitionMat(A, 1.0, 1000))
print(StateTransitionMat(A, 10., 10))
print(StateTransitionMat(A, 10., 100))
print(StateTransitionMat(A, 10., 1000))

[[ 1.01  0.  ]
 [ 0.    1.01]]
[[ 1.01005017  0.        ]
 [ 0.          1.01005017]]
[[ 1.01005017  0.        ]
 [ 0.          1.01005017]]
[[ 1.01005017  0.        ]
 [ 0.          1.01005017]]
[[ 1.10517092  0.        ]
 [ 0.          1.10517092]]
[[ 2.71828183  0.        ]
 [ 0.          2.71828183]]
[[ 12842.30511464      0.        ]
 [     0.          12842.30511464]]
[[ 22026.46579481      0.        ]
 [     0.          22026.46579481]]
[[ 22026.46579481      0.        ]
 [     0.          22026.46579481]]


## 安定性
正方行列$A \in R^{n\times n}$の固有値$\lambda \in C$, 固有ベクトル$v \in R^n$は以下の式を満たす．
$$
Av_i = \lambda_i, i = 1, 2, \cdots, n
$$

固有値$\lambda_i$の実部が全て負であるとき，そのシステムは安定である．

In [20]:
# 固有値・固有ベクトル計算
import numpy.linalg as LA

A = np.array([[1, 2], [3, 4]])
values, vectors = LA.eig(A)

print(values)
print(vectors) 

# Aの固有値手計算すると，
e1 = (5 + np.sqrt(33))/2.
e2 = (5 - np.sqrt(33))/2.
v1 = np.array([1, (3+np.sqrt(33))/4.])
v1 /= LA.norm(v1)

v2 = np.array([1, (3-np.sqrt(33))/4.])
v2 /= LA.norm(v2)

print (e1)
print (e2)
print (v1)
print (v2)

# numpy.linalgの固有ベクトルは縦に並んでいる

[-0.37228132  5.37228132]
[[-0.82456484 -0.41597356]
 [ 0.56576746 -0.90937671]]
5.37228132327
-0.372281323269
[ 0.41597356  0.90937671]
[ 0.82456484 -0.56576746]


## 可制御性
システムが可制御であることの必要十分条件は，可制御性行列$U_c \in R^{n\times nm}$のランクがシステムの次元nと等しくなることである．
$$
U_c = [B\quad AB\quad A^2B\quad \cdots\quad A^{n-1}B]
$$

## 可観測性 
システムが可観測であることの必要十分条件は，可観測性行列$U_o \in R^{rn\times n}$のランクがシステムの次元nと等しくなることである．
$$
U_o = \left[
\begin{array}
CA\\
CA^2\\
\vdots\\
CA^{n-1}
\end{array}
\right]
$$

## フィードバック 
状態変数を入力にフィードバックさせることによる，
* 応答に関わるシステムの極を変更する極配置法
* 状態変数をゼロに整定させるレギュレータ
* 目標値に出力を追従させるサーボ系
といった制御系を構築するための手法が存在する．

状態変数を入力にフィードバックさせるとは，以下のようにフィードバック係数$F \in R^{m\times n}$により，入力を決定することを指す．
$$
u(t) = -Fx(t)
$$

## 最適レギュレータ
評価関数$J$を定義して，リカッチ代数方程式を解くことで，応答と操作量のトレードオフの関係で妥当なフィードバック係数を得る．
$$
J = \frac{1}{2}\int^\infty_0 \left\{x^T(t)Qx(t) + u^T(t)Ru(t) \right\}dt
$$
$Q$は半正定対称行列，$R$は正定対称行列とされ，試行錯誤的に設定される．($Q$, $R$と応答との具体的な関係に非解明な部分？)

この評価関数を最小にするには，以下のリカッチ代数方程式の解$P \in R^{n\times n}$を用いた状態フィードバック制御を行えば良い．
$$
A^{T}P + PA + Q -PBR^{-1}B^{T}P = 0
$$
$P$による状態フィードバック制御は，
$$
u(t) = -Fx(t) = -R^{-1}B^{T}Px(t)
$$
となる．

## サーボ系 
サーボ系とは与えられた目標値に対して，定常偏差がゼロになるような出力を生成させるための誤差システム

## オブザーバ 
状態フィードバックの際には，当然ながら状態変数$x$が必要となる．
しかし，状態変数ベクトル$x$の内，全ての要素が直接観測(例えばセンサなどで読み取る)可能とは限らない．
モデル化されたシステム，即ち状態方程式を用いて，システムが制御されている様子をシミュレートすることで，
直接観測できない状態変数の要素を推定することができる．
全状態を推定するオブザーバと状態のある部分要素のみを推定するオブザーバがある．